# Detecção de Landmarks

Este notebook introduz o conceito de **Detecção de Landmarks (Landmark Detection)**, uma tarefa fundamental em Visão Computacional. O objetivo é identificar e localizar pontos-chave de interesse (landmarks) em um objeto dentro de uma imagem. Esta técnica é a base para muitas aplicações, como rastreamento facial, análise de pose, e reconhecimento de gestos. Inicialmente, construiremos um modelo convolucional simples para encontrar landmarks em um dataset sintético. Em seguida, exploraremos como utilizar bibliotecas de ponta, como o **MediaPipe**, para realizar detecções de landmarks complexas de forma eficiente.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Criação do Dataset Sintético

Para treinar nosso modelo, precisamos de dados. Criaremos um dataset sintético de imagens simples e definiremos os landmarks.

A classe `SyntheticArmDataset` gerará um braço articulado. O processo para gerar cada amostra é:

1.  Definir um ponto de origem (ombro) em uma posição aleatória.
2.  Definir um comprimento (len1) e um ângulo (theta1) aleatórios para o segmento do braço.
3.  Calcular a posição do cotovelo usando trigonometria.
4.  Definir um comprimento (len2) e um ângulo *relativo* (theta2) aleatórios para o antebraço.
5.  Calcular a posição do pulso com base na posição do cotovelo e nos novos parâmetros.
6.  Desenhar os dois segmentos na imagem usando `cv2.line` com espessura para simular cilindros.
7.  Armazenar os 3 landmarks (6 coordenadas) e normalizá-los para o intervalo `[-1, 1]`.

In [ ]:
class SyntheticArmDataset(Dataset):
    def __init__(self, num_samples=1000, img_size=(96, 96), num_landmarks=3):
        self.num_samples = num_samples
        self.img_size = img_size
        self.num_landmarks = num_landmarks
        self.img_height, self.img_width = img_size

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Criar uma imagem preta
        image = np.zeros((self.img_height, self.img_width), dtype=np.float32)
        h, w = self.img_size
        
        # Parâmetros do braço
        segment_thickness = 4
        len1 = np.random.uniform(h * 0.2, h * 0.4) # Comprimento braço
        len2 = np.random.uniform(h * 0.2, h * 0.4) # Comprimento antebraço
        
        # Ângulos
        theta1 = np.random.uniform(0, 2 * math.pi) # Ângulo do braço
        theta2 = np.random.uniform(-math.pi / 1.5, math.pi / 1.5) # Ângulo relativo do cotovelo
        
        # Ponto 1: Ombro (Shoulder)
        # Manter o ombro um pouco longe das bordas
        x_s = np.random.randint(w * 0.2, w * 0.8)
        y_s = np.random.randint(h * 0.2, h * 0.8)
        
        # Ponto 2: Cotovelo (Elbow)
        x_e = x_s + len1 * math.cos(theta1)
        y_e = y_s + len1 * math.sin(theta1)
        
        # Ponto 3: Pulso (Wrist)
        # O ângulo do antebraço é a soma do ângulo do braço + ângulo relativo do cotovelo
        x_w = x_e + len2 * math.cos(theta1 + theta2)
        y_w = y_e + len2 * math.sin(theta1 + theta2)
        
        # Garantir que os pontos estão dentro dos limites da imagem
        points = np.array([x_s, y_s, x_e, y_e, x_w, y_w])
        points = np.clip(points, 0, max(h, w) - 1).astype(int)
        
        (x_s, y_s, x_e, y_e, x_w, y_w) = points
        
        # Desenhar os "cilindros" (linhas grossas)
        cv2.line(image, (x_s, y_s), (x_e, y_e), (255), segment_thickness)
        cv2.line(image, (x_e, y_e), (x_w, y_w), (255), segment_thickness)
        
        # Normalizar imagem
        image = image / 255.0
        image_tensor = torch.from_numpy(image).unsqueeze(0).float() # (1, H, W)
        
        # Preparar landmarks (x1, y1, x2, y2, x3, y3)
        # Usamos os valores *antes* do clip para a regressão,
        # mas convertemos os 'points' (pós-clip) para float32 para normalização.
        landmarks = points.astype(np.float32)
        
        # Normalizar landmarks para [-1, 1]
        landmarks[0::2] = (landmarks[0::2] - w / 2) / (w / 2) # Coordenadas X
        landmarks[1::2] = (landmarks[1::2] - h / 2) / (h / 2) # Coordenadas Y
        
        landmarks_tensor = torch.from_numpy(landmarks)
        
        return image_tensor, landmarks_tensor

In [ ]:
IMG_SIZE = (128, 128)
NUM_LANDMARKS = 3
NUM_OUTPUTS = NUM_LANDMARKS * 2

# Criar datasets e dataloaders
train_dataset = SyntheticArmDataset(num_samples=10000, img_size=IMG_SIZE, num_landmarks=NUM_LANDMARKS)
val_dataset = SyntheticArmDataset(num_samples=500, img_size=IMG_SIZE, num_landmarks=NUM_LANDMARKS)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# Visualizar um exemplo
sample_image, sample_landmarks = train_dataset[0]

# Desnormalizar landmarks para visualização
lm_unnormalized = sample_landmarks.numpy().copy()
lm_unnormalized[0::2] = (lm_unnormalized[0::2] * (IMG_SIZE[1] / 2)) + (IMG_SIZE[1] / 2)
lm_unnormalized[1::2] = (lm_unnormalized[1::2] * (IMG_SIZE[0] / 2)) + (IMG_SIZE[0] / 2)

xs = lm_unnormalized[0::2] # [x_s, x_e, x_w]
ys = lm_unnormalized[1::2] # [y_s, y_e, y_w]

plt.imshow(sample_image.squeeze(), cmap='gray')
plt.scatter(xs, ys, c='red', s=40)
plt.title("Exemplo de Braço Sintético e 3 Landmarks")
plt.show()

## Definição do Modelo CNN

Definimos uma arquitetura de CNN simples. A rede consiste em blocos convolucionais para extração de características, seguidos por camadas totalmente conectadas (Lineares) para regredir as coordenadas dos landmarks.

A camada de saída final deve ter `NUM_OUTPUTS` neurônios, correspondendo às `NUM_LANDMARKS * 2` coordenadas (x, y).

In [ ]:
class LandmarkCNN(nn.Module):
    def __init__(self, img_channels, num_outputs):
        super(LandmarkCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(img_channels, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.AdaptiveAvgPool2d((8, 8)) # (N, 128, 8, 8)
        )
        
        self.flatten_features = 128 * 8 * 8
        
        self.regressor = nn.Sequential(
            nn.Linear(self.flatten_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_outputs)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.regressor(x)
        return x

model = LandmarkCNN(img_channels=1, num_outputs=NUM_OUTPUTS).to(device)
print(f"Modelo criado com {NUM_OUTPUTS} saídas.")
print(model)

## Treinamento do Modelo

Para treinar o modelo, precisamos definir uma função de perda (Loss Function) e um otimizador.

### Função de Perda

Como esta é uma tarefa de regressão (prever coordenadas), a função de perda apropriada é o **Erro Quadrático Médio (Mean Squared Error - MSE)**. O MSE calcula a média das diferenças quadráticas entre as coordenadas previstas e as coordenadas reais (ground truth).

$$
\text{MSE} = \frac{1}{N} \sum_{i=1}^{N} \frac{1}{k} \sum_{j=1}^{k} (y_{ij} - \hat{y}_{ij})^2
$$

Onde:
$N$ é o número de amostras no batch.
$k$ é o número de valores de saída (nossos `NUM_OUTPUTS`).
$y_{ij}$ é a j-ésima coordenada real para a i-ésima amostra.
$\hat{y}_{ij}$ é a j-ésima coordenada prevista para a i-ésima amostra.

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for images, landmarks in train_loader:
        images = images.to(device)
        landmarks = landmarks.to(device)
        
        # Zerar gradientes
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, landmarks)
        
        # Backward pass e otimização
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        
    epoch_loss = running_loss / len(train_dataset)
    
    # Validação
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, landmarks in val_loader:
            images = images.to(device)
            landmarks = landmarks.to(device)
            outputs = model(images)
            loss = criterion(outputs, landmarks)
            val_loss += loss.item() * images.size(0)
            
    val_loss = val_loss / len(val_dataset)
    
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.6f}, Val Loss: {val_loss:.6f}')

## Visualização dos Resultados

Vamos agora pegar algumas amostras do conjunto de validação, passar pelo modelo treinado e visualizar as previsões de landmarks em comparação com o ground truth.

In [ ]:
def visualize_predictions(model, dataset, num_samples=5):
    model.eval()
    fig, axes = plt.subplots(1, num_samples, figsize=(15, 5))
    
    for i in range(num_samples):
        # Pegar amostra
        image_tensor, gt_landmarks = dataset[i]
        image_tensor_batch = image_tensor.unsqueeze(0).to(device) # Adicionar dimensão do batch
        
        # Fazer previsão
        with torch.no_grad():
            pred_landmarks = model(image_tensor_batch).cpu().squeeze()
            
        # Preparar imagem para plotagem
        image = image_tensor.squeeze().numpy()
        
        # Desnormalizar Ground Truth (shape [6])
        gt_coords = gt_landmarks.numpy().copy()
        gt_xs = (gt_coords[0::2] * (IMG_SIZE[1] / 2)) + (IMG_SIZE[1] / 2)
        gt_ys = (gt_coords[1::2] * (IMG_SIZE[0] / 2)) + (IMG_SIZE[0] / 2)
        
        # Desnormalizar Previsões (shape [6])
        pred_coords = pred_landmarks.detach().numpy().copy()
        pred_xs = (pred_coords[0::2] * (IMG_SIZE[1] / 2)) + (IMG_SIZE[1] / 2)
        pred_ys = (pred_coords[1::2] * (IMG_SIZE[0] / 2)) + (IMG_SIZE[0] / 2)

        # Plotar
        ax = axes[i]
        ax.imshow(image, cmap='gray', interpolation=None)
        ax.scatter(gt_xs, gt_ys, c='red', s=50, label='Ground Truth', marker='x')
        ax.scatter(pred_xs, pred_ys, c='blue', s=50, label='Prediction', marker='+')
        ax.set_title(f"Amostra {i}")
        ax.set_xticks([])
        ax.set_yticks([])

    handles, labels = ax.get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', ncol=2)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Ajustar para a legenda
    plt.show()

# Visualizar nos dados de validação
visualize_predictions(model, val_dataset)

# Detecção de Landmarks com MediaPipe

Construir e treinar modelos de landmarks do zero é instrutivo, mas para aplicações do mundo real, muitas vezes utilizamos soluções pré-treinadas que são altamente otimizadas e precisas. O **MediaPipe** do Google é uma biblioteca de código aberto fantástica para pipelines de ML multimodais (vídeo, áudio, texto).

Ele oferece modelos robustos e em tempo real para tarefas como detecção de face (Face Mesh), mãos (Hand Tracking) e pose (Pose Estimation).

## Instalação

Inicialmente, instalamos as dependências necessárias (`opencv-python`, `mediapipe`) e baixamos as imagens de exemplo (`person-full-body.jpg`, `hand.jpg`) que serão utilizadas.

In [ ]:
# !pip install opencv-python
# !pip install mediapipe

In [ ]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import numpy as np

# Silenciar avisos do MediaPipe (opcional)
import warnings
warnings.filterwarnings('ignore')

## Detecção de Pose (Corpo Inteiro)

Nesta seção, inicializamos a solução `mp.solutions.pose`. Este modelo detecta 33 landmarks do corpo.

In [ ]:
# Carregar módulos específicos de Pose
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
# Carregar a imagem
image_pose = cv2.imread('pose.jpg')
image_pose = cv2.cvtColor(image_pose, cv2.COLOR_BGR2RGB)

In [ ]:
# Inicializar o modelo e processar a imagem
with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5) as pose:

    # Processar
    results = pose.process(image_pose)

    # Desenhar os landmarks
    annotated_image = image_pose.copy()
    mp_drawing.draw_landmarks(
        annotated_image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2),
        connection_drawing_spec=mp_drawing.DrawingSpec(color=(0,255,0), thickness=2)
    )
    
    # Exibir resultado
    plt.figure(figsize=(10, 8))
    plt.imshow(annotated_image)
    plt.title('Detecção de Pose')
    plt.axis('off')
    plt.show()

## Detecção de Mãos

Agora, usamos a solução `mp.solutions.hands`. Este modelo detecta 21 landmarks para cada mão.

In [ ]:
# Carregar módulos específicos de Mãos
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [ ]:
# Carregar a imagem
image_hand = cv2.imread('hand.jpg')
image_hand = cv2.cvtColor(image_hand, cv2.COLOR_BGR2RGB)

In [ ]:
# Inicializar o modelo e processar
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
    
    # Processar
    results = hands.process(image_hand)

    # Desenhar landmarks
    annotated_image = image_hand.copy()
    for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            annotated_image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style()
        )
    
    # Exibir resultado
    plt.figure(figsize=(10, 8))
    plt.imshow(annotated_image)
    plt.title('Detecção de Mãos')
    plt.axis('off')
    plt.show()

## Detecção de Malha Facial (Face Mesh)

Por fim, usamos `mp.solutions.face_mesh` para detectar 478 landmarks faciais (incluindo íris, se `refine_landmarks=True`).

In [ ]:
# Carregar módulos específicos de Face Mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

In [ ]:
# Carregar a imagem
image_face = cv2.imread('face.jpg')
image_face = cv2.cvtColor(image_face, cv2.COLOR_BGR2RGB)

In [ ]:
# Inicializar o modelo e processar
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5) as face_mesh:
    
    # Processar
    results = face_mesh.process(image_face)

    # Desenhar landmarks
    annotated_image = image_face.copy()
    for face_landmarks in results.multi_face_landmarks:

        for face_landmarks in results.multi_face_landmarks:
            h, w, _ = image_face.shape
            x_points = [lm.x * w for lm in face_landmarks.landmark]
            y_points = [lm.y * h for lm in face_landmarks.landmark]
            
            plt.figure(figsize=(10, 8))
            plt.imshow(image_face)
            plt.scatter(x_points, y_points, s=5, c='lime')
            plt.title('Landmarks Faciais - Apenas Pontos')
            plt.axis('off')
            plt.show()
        
        # 1. Tessellation (malha completa)
        mp_drawing.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style()
        )

        # 2. Contornos (linhas do rosto)
        mp_drawing.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style()
        )

        # 3. Íris
        mp_drawing.draw_landmarks(
            image=annotated_image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_IRISES,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_iris_connections_style()
        )

    # Exibir resultado
    plt.figure(figsize=(10, 8))
    plt.imshow(annotated_image)
    plt.title('Detecção de Malha Facial (Contornos e Íris)')
    plt.axis('off')
    plt.show()